# References:

https://www.sbert.net/examples/applications/semantic-search/README.html

https://www.sbert.net/docs/pretrained_models.html

In this notebook, I loaded datasets containing training, evaluation, and test samples for a question answering task. After extracting questions and contexts from the training and evaluation datasets, I utilized the SentenceTransformer library to load a pre-trained model for encoding text into embeddings. Using this model, I encoded the training and evaluation questions, normalized the embeddings, and employed semantic search to find the most similar question in the training set for each evaluation question. Subsequently, I calculated the accuracy of this model in identifying the correct context for evaluation questions and determined the best-performing model based on accuracy. Furthermore, I authenticated with the Hugging Face Hub and imported necessary libraries to work with a specific pre-trained model called Mistral AI `mistralai/Mistral-7B-v0.1`. After defining its configuration and loading its weights for causal language modeling, I loaded a previously trained and fine-tuned model from a checkpoint directory. To ensure reproducibility, I set a random seed and selected a random test sample from the test dataset. Then, I encoded the test query, found the most similar question in the training set, and retrieved its context. Using this information, I created a prompt and tokenized it for model input. Finally, I generated text based on the model input, disabled gradient calculation during inference, and decoded the generated text to produce a response.

# Load Datasets

In [6]:
from datasets import load_dataset

# Load training dataset
train_dataset = load_dataset('json', data_files='../data/train_CRM_data.json', split='train')

# Load evaluation dataset
eval_dataset = load_dataset('json', data_files='../data/val_CRM_data.json', split='train')

# Load test dataset
test_dataset = load_dataset('json', data_files='../data/test_CRM_data.json', split='train')

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

# Extract Questions and Contexts

In [7]:
def create_question_context_dict(message):
    """
    Create a dictionary with question as key and context as value

    Args:
        message (list): A list containing dictionaries for question, context, and answer.

    Returns:
        dict: Dictionary of question as key and context as value
    """
    # Extracting question, context, and answer from the message
    question = message[1]['content']
    context = message[0]['content']
    answer = message[2]['content']

    return {question: context}

In [8]:
train_question_context_dict = {}

# Iterate through each example in the training dataset
for sample in train_dataset:
    
    # Extract question and context using create_question_context_dict function
    qc_pair = create_question_context_dict(sample['messages'])
    
    # Merge the extracted pairs into the train_question_context_dict
    train_question_context_dict.update(qc_pair)

eval_question_context_dict = {}

# Iterate through each example in the evaluation dataset
for sample in eval_dataset:
    
    # Extract question and context using create_question_context_dict function
    qc_pair = create_question_context_dict(sample['messages'])
    
    # Merge the extracted pairs into the eval_question_context_dict
    eval_question_context_dict.update(qc_pair)

# Load SentenceTransformer Model, Encode Training Questions, Encode Evaluation Questions

In [4]:
from sentence_transformers import SentenceTransformer, util
import torch

# Load pre-trained SentenceTransformer model
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Extracting all questions from train_question_context_dict
train_questions_corpus = list(train_question_context_dict.keys())

# Encode the questions into embeddings
train_corpus_embeddings = embedder.encode(train_questions_corpus, convert_to_tensor=True)

# Move embeddings to GPU if available
train_corpus_embeddings = train_corpus_embeddings.to("cuda")

# Normalize embeddings
train_corpus_embeddings = util.normalize_embeddings(train_corpus_embeddings)

# Query sentences:
eval_questions_queries = list(eval_question_context_dict.keys())

# Encode evaluation questions into embeddings
query_embeddings = embedder.encode(eval_questions_queries, convert_to_tensor=True)

# Move query embeddings to GPU if available
query_embeddings = query_embeddings.to("cuda")

# Normalize query embeddings
query_embeddings = util.normalize_embeddings(query_embeddings)

# Find the most similar question in the training set for each evaluation question
hits = util.semantic_search(query_embeddings, train_corpus_embeddings, score_function=util.dot_score, top_k=1)

# Find Similar Questions

In [5]:
import pandas as pd

output_data = []

# Iterate through each evaluation question and its corresponding hit
for idx, eval_questions_query in enumerate(eval_questions_queries):
    eval_question = eval_questions_query
    
    # Get the best matching train question using the hits
    best_matching_train_question = train_questions_corpus[hits[idx][0]['corpus_id']]
    
    # Check if the correct context is identified by comparing the contexts of the best matching train question and evaluation question
    correct_context_identified = train_question_context_dict[best_matching_train_question] == eval_question_context_dict[eval_questions_query]
    
    # Append the data to the output list
    output_data.append({
        'Eval Question': eval_question,
        'Best Matching Train Question': best_matching_train_question,
        'Correct Context Identified': correct_context_identified
    })

# Creating a DataFrame from the output data
output_df = pd.DataFrame(output_data)

# Displaying the DataFrame
print(output_df)

                                        Eval Question  \
0   What is the correlation between transaction sc...   
1   Most Common Product Category Mentioned in Cust...   
2   Are there any products with a spike in transac...   
3   Could you evaluate the engagement predictions ...   
4   Can you identify any outliers in transaction s...   
5   Are there any products with consistently low t...   
6          Can you identify the top-selling products?   
7   How many transactions have occurred for each c...   
8   Are there any products with a consistent incre...   
9   Are there any trends or patterns in purchasing...   
10  Are there any specific customer segments that ...   
11  How many transactions in our database have a s...   
12  Are there any products that are frequently pur...   
13  Are there any products with a spike in transac...   
14  Are there any trends in the rankings of produc...   
15  Are there any outliers in terms of high-volume...   
16  Are there any external fact

# Calculate Accuracy

In [6]:
# Calculate the total number of evaluations
total_evaluations = len(output_df)

# Calculate the number of correct context identifications
correct_identifications = output_df['Correct Context Identified'].sum()

# Calculate accuracy by dividing the number of correct identifications by the total evaluations
accuracy = correct_identifications / total_evaluations

# Print the accuracy
print("Accuracy:", accuracy)

Accuracy: 0.6585365853658537


# Identify Best Model

In [7]:
from sentence_transformers import SentenceTransformer, util
import torch
import pandas as pd

def evaluate_model_accuracy(model_name, train_question_context_dict, eval_question_context_dict):
    """
    Evaluate the accuracy of a sentence transformer model on a given set of training and evaluation question-context pairs.

    Args:
        model_name (str): Name of the sentence transformer model to be used for embedding.
        train_question_context_dict (dict): Dictionary mapping training questions to their corresponding contexts.
        eval_question_context_dict (dict): Dictionary mapping evaluation questions to their corresponding contexts.

    Returns:
        float: Accuracy of the model in identifying the correct context for evaluation questions.
    """
    # Load the model
    embedder = SentenceTransformer(model_name)

    # Extracting all questions from train_question_context_dict
    train_questions_corpus = list(train_question_context_dict.keys())
    
    # Encode the training questions into embeddings and move them to GPU if available
    train_corpus_embeddings = embedder.encode(train_questions_corpus, convert_to_tensor=True).to("cuda")
    
    # Normalize the embeddings
    train_corpus_embeddings = util.normalize_embeddings(train_corpus_embeddings)

    # Query sentences:
    eval_questions_queries = list(eval_question_context_dict.keys())
    
    # Encode the evaluation questions into embeddings and move them to GPU if available
    query_embeddings = embedder.encode(eval_questions_queries, convert_to_tensor=True).to("cuda")
    
    # Normalize the embeddings
    query_embeddings = util.normalize_embeddings(query_embeddings)

    # Find the most similar question in the training set for each evaluation question
    hits = util.semantic_search(query_embeddings, train_corpus_embeddings, score_function=util.dot_score, top_k=1)

    output_data = []
    for idx, eval_questions_query in enumerate(eval_questions_queries):
        eval_question = eval_questions_query
        
        # Get the best matching train question using the hits
        best_matching_train_question = train_questions_corpus[hits[idx][0]['corpus_id']]
        
        # Check if the correct context is identified by comparing the contexts of the best matching train question and evaluation question
        correct_context_identified = train_question_context_dict[best_matching_train_question] == eval_question_context_dict[eval_questions_query]

        output_data.append({
            'Eval Question': eval_question,
            'Best Matching Train Question': best_matching_train_question,
            'Correct Context Identified': correct_context_identified
        })

    # Creating a DataFrame from the output data
    output_df = pd.DataFrame(output_data)

    # Calculate the total number of evaluations
    total_evaluations = len(output_df)

    # Calculate the number of correct context identifications
    correct_identifications = output_df['Correct Context Identified'].sum()

    # Calculate accuracy
    accuracy = correct_identifications / total_evaluations

    return accuracy

# List of models to evaluate
model_names = [
    "all-mpnet-base-v2",
    "gtr-t5-xxl", 
    "gtr-t5-xl", 
    "sentence-t5-xxl",
    "gtr-t5-large",
    "all-mpnet-base-v1",
    "multi-qa-mpnet-base-dot-v1",
    "multi-qa-mpnet-base-cos-v1",
    "all-roberta-large-v1",
    "sentence-t5-xl",
    "all-distilroberta-v1",
    "all-MiniLM-L12-v1",
    "all-MiniLM-L12-v2",
    "multi-qa-distilbert-dot-v1",
    "multi-qa-distilbert-cos-v1",
    "gtr-t5-base",
    "sentence-t5-large",
    "all-MiniLM-L6-v2",
    "multi-qa-MiniLM-L6-cos-v1",
    "all-MiniLM-L6-v1",
    "paraphrase-mpnet-base-v2",
    "msmarco-bert-base-dot-v5",
    "multi-qa-MiniLM-L6-dot-v1",
    "sentence-t5-base",
    "msmarco-distilbert-base-tas-b",
    "msmarco-distilbert-dot-v5",
    "paraphrase-distilroberta-base-v2",
    "paraphrase-MiniLM-L12-v2",
    "paraphrase-multilingual-mpnet-base-v2",
    "paraphrase-TinyBERT-L6-v2",
    "paraphrase-MiniLM-L6-v2",
    "paraphrase-albert-small-v2",
    "paraphrase-multilingual-MiniLM-L12-v2",
    "paraphrase-MiniLM-L3-v2",
    "distiluse-base-multilingual-cased-v1",
    "distiluse-base-multilingual-cased-v2",
    "average_word_embeddings_komninos",
    "average_word_embeddings_glove.6B.300d"
]

# Dictionary to store accuracy results
accuracy_results = {}

# Evaluate each model and store accuracy results
for model_name in model_names:
    accuracy = evaluate_model_accuracy(model_name, train_question_context_dict, eval_question_context_dict)
    accuracy_results[model_name] = accuracy

# Print accuracy results
for model_name, accuracy in accuracy_results.items():
    print(f"Model: {model_name}, Accuracy: {accuracy}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

/home/jovyan/LLM_CRM_PERSO/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1006: UserWarning: Not enough free disk space to download the file. The expected file size is: 669.90 MB. The target location /home/jovyan/.cache/huggingface/hub only has 496.44 MB free disk space.
  warnings.warn(
/home/jovyan/LLM_CRM_PERSO/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1006: UserWarning: Not enough free disk space to download the file. The expected file size is: 669.90 MB. The target location /home/jovyan/.cache/huggingface/hub/models--sentence-transformers--gtr-t5-large/blobs only has 496.44 MB free disk space.
  warnings.warn(


model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

OSError: [Errno 28] No space left on device

In [8]:
# Find the model with the highest accuracy
best_model = max(accuracy_results, key=accuracy_results.get)

# Get the accuracy of the best model
best_accuracy = accuracy_results[best_model]

# Print the best model and its accuracy
print(f"The best model based on accuracy is '{best_model}' with an accuracy of {best_accuracy:.2%}.")

The best model based on accuracy is 'all-mpnet-base-v2' with an accuracy of 73.17%.
The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


# Authenticate Hugging Face Hub

In [2]:
# Import the notebook_login function from huggingface_hub module
from huggingface_hub import notebook_login

# Call the notebook_login function to authenticate
# Provide an access token from the provided URL - https://huggingface.co/settings/tokens
notebook_login()

# Import Libraries and Load Pretrained Model

In [3]:
# Import necessary libraries
from sentence_transformers import SentenceTransformer, util
import torch
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    BitsAndBytesConfig
)

# Define Mistral's pretrained model ID
base_model_id = "mistralai/Mistral-7B-v0.1"

# Define BitsAndBytesConfig for Mistral's model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                    # Load the model in 4-bit precision
    bnb_4bit_use_double_quant=True,       # Use double quantization for 4-bit quantization
    bnb_4bit_quant_type="nf4",            # Use nf4 quantization for 4-bit quantization
    bnb_4bit_compute_dtype=torch.bfloat16 # Use bfloat16 for computation with 4-bit quantization
)

# Load Mistral's pretrained model for causal language modeling
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id, 
    quantization_config=bnb_config, # Apply the defined quantization configuration
    device_map="auto"               # Automatically select the device for model inference
)

# Load Mistral's tokenizer
eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id, 
    add_bos_token=True,     # Add beginning-of-sequence token
    trust_remote_code=True  # Trust remote code for tokenization
)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

# Load Fine-Tuned Model

In [4]:
from peft import PeftModel

# Load the weights from the checkpoint directory
ft_model = PeftModel.from_pretrained(base_model, "../models/01-finetune-mistral/checkpoint-150")

# Retrieve context for a random test sample question and prepare prompt

In [13]:
import random

# Set the seed for reproducibility
random.seed(42)

# Take a random index
random_index = random.randint(0, len(test_dataset) - 1)

# Take the random sample
random_test_sample = test_dataset[random_index]

embedder = SentenceTransformer("all-mpnet-base-v2")

# Extracting all questions from train_question_context_dict
train_questions_corpus = list(train_question_context_dict.keys())

# Encode the training questions into embeddings and move them to GPU if available
train_corpus_embeddings = embedder.encode(train_questions_corpus, convert_to_tensor=True).to("cuda")

# Normalize the embeddings
train_corpus_embeddings = util.normalize_embeddings(train_corpus_embeddings)

# Query sentence:
test_query = [random_test_sample['messages'][1]['content']]

# Encode the test query into embeddings and move them to GPU if available
query_embedding = embedder.encode(test_query, convert_to_tensor=True).to("cuda")

# Normalize the embeddings
query_embedding = util.normalize_embeddings(query_embedding)

# Find the most similar question in the training set for the test query
hits = util.semantic_search(query_embedding, train_corpus_embeddings, score_function=util.dot_score, top_k=1)

# Retrieve the context corresponding to the most similar question
retrieved_context = train_question_context_dict[train_questions_corpus[hits[0][0]['corpus_id']]]

# Create a prompt with the test query and the retrieved context
prompt = f"""
### Question: {test_query[0]}

### Context: {retrieved_context}
### Answer:
""".strip()

# Init a tokenizer that doesn't add padding or eos token
test_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
)

# Tokenize the test prompt and prepare model input for inference
model_input = test_tokenizer(
    prompt, 
    return_tensors="pt"
).to("cuda")  # Move tensors to GPU if available

# Set the fine-tuned model to evaluation mode
ft_model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Lin

# Model Inferencing

In [14]:
# Disable gradient calculation during inference
with torch.no_grad():
    # Generate text based on the model input
    generated_tokens = ft_model.generate(
        **model_input,                            # Pass model input
        max_new_tokens=1024,                      # Maximum number of new tokens to generate
        repetition_penalty=1.15,                  # Repetition penalty to avoid repetition
        pad_token_id=eval_tokenizer.eos_token_id  # Set pad token ID
    )[0]                                          # Get the first generated sequence

    # Decode the generated tokens into text, skipping special tokens
    generated_text = eval_tokenizer.decode(
        generated_tokens,         # Generated tokens
        skip_special_tokens=True  # Skip special tokens like padding and eos
    )

    # Print the generated text
    print(generated_text)

### Question: What is the average transaction score for each product category?

### Context: 
You are a Python function generator. Users will ask you questions in English, 
and you will produce a Python function as answer to the question based on the provided CONTEXT.

CONTEXT:
Pandas DataFrame df containing transaction data with columns order_id, user_id, item_id, timestamp, score.
order_id takes string datatype and identifies the order.
user_id takes string datatype and identifies the customer.
item_id takes string datatype that identifies the product.
timestamp takes timestamp datatype and represents the datetimestamp of transaction.
score takes float datatype and represents the score of the transaction.
Note that a customer can make multiple transactions for a product but the customer product pair will be just one entry for each order.
Pandas DataFrame customer_dfcontaining customer data with columns user_id, customer_city.
user_id takes string datatype and identifies the customer.

In [15]:
random_test_sample # Original test message

{'messages': [{'content': '\nYou are a Python function generator. Users will ask you questions in English, \nand you will produce a Python function as answer to the question based on the provided CONTEXT.\n\nCONTEXT:\nPandas DataFrame df containing transaction data with columns order_id, user_id, item_id, timestamp, score.\r\norder_id takes string datatype and identifies the order.\r\nuser_id takes string datatype and identifies the customer.\r\nitem_id takes string datatype that identifies the product.\r\ntimestamp takes timestamp datatype and represents the datetimestamp of transaction.\r\nscore takes float datatype and represents the score of the transaction.\r\nNote that a customer can make multiple transactions for a product but the customer product pair will be just one entry for each order.\r\nPandas DataFrame customer_dfcontaining customer data with columns user_id, customer_city.\r\nuser_id takes string datatype and identifies the customer.\r\ncustomer_city takes string dataty

Retrieval Augmented Generation (RAG) indeed leverages semantic search to enhance text generation by retrieving relevant context from a large database. This approach ensures that generated text is more accurate and contextually appropriate. Using powerful VectorDBs can certainly scale up the performance of semantic search, making it more efficient and effective in handling large volumes of data. This combination of techniques holds a lot of promise for improving natural language generation tasks across various domains.